In [1]:
import sys
sys.path.append('/home/ian/code/github')

## 获取症状列表

In [2]:
%%time
from model import model_wrapper

CPU times: user 658 ms, sys: 98.9 ms, total: 757 ms
Wall time: 767 ms


In [3]:
%%time
model = model_wrapper()

5898 ['脑血管痉挛', '脑干出血', '多梦', '失眠', '脑缺氧']
CPU times: user 45.6 s, sys: 2.19 s, total: 47.8 s
Wall time: 47.9 s


In [5]:
model.get_zzs('我牙肿了')

牙肿


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:71: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['牙周脓肿', '水肿', '瘀肿', '脐疮', '脓肿']

In [8]:
model.get_zzs('我牙出血了')

牙出血


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:71: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['刷牙出血', '肠出血', '牙龈出血', '痔出血']

In [8]:
model.get_zzs('我有点发烧')

发烧


'发烧'

In [10]:
model.get_zzs('我睡不着觉')

睡不着觉


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:70: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['梦幻觉', '失眠', '饮不解渴', '头昏想睡觉', '气不摄血']

In [12]:
model.get_zzs('我感觉头有点晕')

头晕


'头晕'

In [13]:
model.get_zzs('我感觉胳膊有点疼痛')

上臂上肢前臂疼痛


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:70: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['腕部疼痛', '膝前疼痛', '膝关节疼痛', '前臂和腕部的疼痛', '伸臂握手疼痛']

In [14]:
model.get_zzs('我感觉肚子有点痛')

肚子腹部痛


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:70: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['脚趾痛', '急性腹痛', '上腹部压痛', '下腹痛', '孕妇肚子痒']

In [16]:
model.get_zzs('我感觉脚有点痛')

脚足痛


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:70: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['脑血管痉挛', '脑干出血', '脚趾痛', '面痛', '足痛']

In [17]:
model.get_zzs('我感觉小腿有点疼')

小腿下肢疼


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:70: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['小腿酸痛', '小腿发胀', '脚后跟疼', '小腿以下缺血', '小腿水肿']

In [18]:
model.get_zzs('我感觉有点恶心')

恶心


'恶心'

In [19]:
model.get_zzs('我感觉有点想吐')

想吐


/home/ian/code/github/crawl/requestProj/39jiankang/model.py:70: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(v1, v2) / (norm(v1) * norm(v2))


['麻痹', '吐弄舌', '吐泡沫痰', '耸肩', '脉数']

In [7]:
model.zz2zz(['头胀'])

(['头痛', '血压高', '头晕', '红细胞增多', '听力减退', '头昏', '发作性眩晕', '乏力', '单侧头痛', '健忘'],
 [4.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [81]:
# from utils.pymongo_wrapper import pymongo_wrapper as pmw

# client = pmw()

# zztable = client.get_collection('jiankang39', 'zz')

# zzlist = [i['症状名称'] for i in client.find_all(zztable, fieldlist=['症状名称'])]



# from utils.pickle_wrapper import pickle_wrapper as picklew

# picklew.dump2file(zzlist,'症状列表.pkl')
from utils.pickle_wrapper import pickle_wrapper as picklew
zzlist = picklew.loadfromfile('症状列表.pkl')
len(zzlist),zzlist[:5]

(5898, ['脑血管痉挛', '脑干出血', '多梦', '失眠', '脑缺氧'])

### 通过编辑距离计算相似度

In [4]:
from utils.nlp.nlp_wrapper import nlp_wrapper as nlpw
from utils.pandas_wrapper import pandas_wrapper as pdw
def get_zzs_by_editdis(s1,zzlist, n=3):
    e = list(map(nlpw.cal_editdistance, [s1]*len(zzlist), zzlist))
    df = pdw.build_df_with_dict({'zz':zzlist,'e':e})
    return list(pdw.sort_by_column(df,'e').loc[:,'zz'][:n].values)

In [6]:
get_zzs_by_editdis('睡不着觉',zzlist)

['气不摄血', '不排卵', '不举']

In [7]:
get_zzs_by_editdis('我经常失眠',zzlist)

['失眠', '色觉丧失', '绝经期面红']

### 通过word2vec模糊推荐症状

In [8]:
import gensim
import jieba
import numpy as np
from scipy.linalg import norm
 
model_file = '/home/ian/code/github/data/word2vec/news_12g_baidubaike_20g_novel_90g_embedding_64.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_file, binary=True)


In [13]:
def vector_similarity(s1, s2):
    def sentence_vector(s):
        words = jieba.lcut(s)
        v = np.zeros(64)
        for word in words:
            if word in model:
                v += model[word]
        v /= len(words)
        return v
    try:
        v1, v2 = sentence_vector(s1), sentence_vector(s2)
        return np.dot(v1, v2) / (norm(v1) * norm(v2))
    except:
        return 0

In [14]:
def get_zzs_by_word2vec(s1,zzlist,n=2):
    e = list(map(vector_similarity, [s1]*len(zzlist), zzlist))
    df = pdw.build_df_with_dict({'zz':zzlist,'e':e})
    return list(pdw.sort_by_column(df,'e',ascending=False).loc[:,'zz'][:n].values)

In [15]:
get_zzs_by_word2vec('睡不着觉',zzlist)

/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['头昏想睡觉', '失眠']

In [16]:
get_zzs_by_word2vec('我经常失眠',zzlist)

/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['经常流鼻血', '头昏想睡觉']

### 得到模糊症状匹配
如何提高准确率？  
* 在模型使用过程中，排除影响模型的个别word  
* zzlist  
保证zzlist都是可问的，就是说根据用户的特征（孕妇，老人，男女）有针对的zzlist

In [74]:
import re
def get_zzs(s1,zzlist):
    s1 = re.sub(r'[我你他很(非常)(有点)的,，。]|(感觉)','',s1)
    s1 = re.sub(r'胳膊','上臂上肢前臂',s1)
    s1 = re.sub(r'小腿','小腿下肢',s1)
    s1 = re.sub(r'大腿','大腿下肢',s1)
    s1 = re.sub(r'脸','脸面部',s1)
    s1 = re.sub(r'喉咙','喉咙咽喉',s1)
    s1 = re.sub(r'嘴','嘴口',s1)
#     s1 = re.sub(r'牙','牙牙齿',s1)
    s1 = re.sub(r'屁股','屁股臀部',s1)
    s1 = re.sub(r'肚子','肚子腹部',s1)
    s1 = re.sub(r'脚','脚足',s1)
#     s1 = re.sub(r'[疼痛]','疼痛',s1)
    print(s1)
    if s1 in zzlist:
        return s1
    return list(set(get_zzs_by_word2vec(s1,zzlist)+get_zzs_by_editdis(s1,zzlist)))

In [56]:
get_zzs('我睡不着觉',zzlist)

睡不着觉


/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['气不摄血', '失眠', '不排卵', '不举', '头昏想睡觉']

In [57]:
get_zzs('我脸有点肿',zzlist)

脸面部肿


/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['痛性脸抽搐', '面部水肿', '囊肿', '面部浮肿', '脸冻伤']

In [64]:
get_zzs('我牙很肿',zzlist)

牙肿


/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['牙疼', '牙周脓肿', '水肿', '瘀肿', '脐疮']

In [65]:
get_zzs('我有点发烧',zzlist)

发烧


'发烧'

In [66]:
get_zzs('我感觉头有点晕',zzlist)

头晕


'头晕'

In [70]:
get_zzs('我感觉胳膊有点疼痛',zzlist)

上臂上肢前臂疼痛


/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['腕部疼痛', '前臂外伤后疼痛', '膝关节疼痛', '膝前疼痛', '前臂和腕部的疼痛']

In [71]:
get_zzs('我感觉小腿有点疼',zzlist)

小腿下肢疼


/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['脚后跟疼', '小腿以下缺血', '小腿酸痛', '小腿水肿']

In [72]:
get_zzs('我感觉大腿有点疼',zzlist)

大腿下肢疼


/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['大腿刺痛', '脚后跟疼', '大腿外侧痛', '小腿酸痛']

In [75]:
get_zzs('我感觉喉咙有点疼',zzlist)

喉咙咽喉疼


/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['喉咙冒黏液', '颈动脉痛', '喉咙痒', '咽喉疼痛']

In [78]:
get_zzs('我感觉脚有点痛',zzlist)

脚足痛


/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['', '右胸痛', '脑血管痉挛', '脚趾痛', '足痛']

In [79]:
get_zzs('我感觉肚子有点痛',zzlist)

肚子腹部痛


/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


['孕妇肚子痒', '运动腹痛', '脚趾痛', '下腹胀痛', '肚子疼']

In [46]:
e = list(map(vector_similarity, ['我牙疼']*len(zzlist), zzlist))
e[:5]

/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


[0.25886484774584,
 0.20469593383021958,
 0.09579593970598164,
 0.2230319561722977,
 0.09064157739034769]

In [47]:
df = pdw.build_df_with_dict({'zz':zzlist,'e':e})
list(pdw.sort_by_column(df,'e',ascending=False).loc[:,'zz'][:5].values)

['双眼难以忍受的奇痒', '上呼吸道卡他症状', '日轻夜重的瘙痒', '反复发作的带状疱疹', '足的放射性疼痛']

In [40]:
pdw.sort_by_column(df,'e')

,zz,e
541,眼底发现黄斑呈带灰色变色,-0.268249
2805,足跟蓝黑色或黑色斑点,-0.261248
2266,诊断室中所测血压始终增高,-0.258950
776,鼻唇沟变浅,-0.245501
754,鼻中隔向一侧或两则弯曲,-0.244906
1889,蓝色、蓝灰色或蓝黑色丘疹,-0.242348
1722,皮肤水疱，后形成黑色干痂,-0.241613
2934,骶部可有皮赘，形成尾巴,-0.239277
1044,面容呈灰色,-0.239224
1533,成人大便绿色,-0.237868
